In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [2]:
from test_model import test
from argparse import Namespace
import glob
import numpy as np
from concurrent import futures
import multiprocessing
import copy
import json

In [3]:
args = {
    'data_path': '../../Data/',
    'pretrained_model': 'incep', #supcon', #'resnet',
    'pretrained_path': '',
    'window_size': 29,
    'strided': 15,
    'batch_size': 512,
    'num_workers': 8,
    'trial_id': 1,
    'ckpt' : 50,
    'car_model': None
}

args = Namespace(**args)

In [4]:
bs = 512
pretrained_paths = sorted(glob.glob(f'../save/inception*'))
# pretrained_paths = sorted(glob.glob(f'../save/SupCon.resnet18*_bs{bs}*'))
# pretrained_paths = sorted(glob.glob('../save/smallresnet18.ce?_*_lr0.001_*'))

pretrained_paths

['../save/inception1_gamma0_lr0.001_bs256_100epochs_092722_095310',
 '../save/inception2_gamma0_lr0.001_bs256_100epochs_092722_143206',
 '../save/inception3_gamma0_lr0.001_bs256_100epochs_092722_143213',
 '../save/inception4_gamma0_lr0.001_bs256_100epochs_092722_143222',
 '../save/inception5_gamma0_lr0.001_bs256_100epochs_092722_143237']

In [5]:
num_cpu = multiprocessing.cpu_count()
workers = max(num_cpu, len(pretrained_paths))

with futures.ProcessPoolExecutor(workers) as executor:
    to_do = []
    for path in pretrained_paths:
        args.trial_id = path.split('_')[0][-1]
        args.pretrained_path = f'{path}/models/'
        future = executor.submit(test, copy.deepcopy(args), 
                                 {'verbose': False, 'is_cuda': True})
        to_do.append(future)
        
    
    total_results = {}
    for future in futures.as_completed(to_do):
        try:
            results = future.result()
            for k, v in results.items():
                total_results.setdefault(k, [])
                total_results[k].append(v)
        except Exception as error:
            print('An exception occurred: {}'.format(error))

Loading:  SupIncepResnet
Loading:  SupIncepResnetLoading: 
Loading:  SupIncepResnet 
SupIncepResnet
Loading:  SupIncepResnet


  0%|          | 0/648 [00:00<?, ?it/s]

  0%|          | 0/648 [00:00<?, ?it/s]

  0%|          | 0/648 [00:00<?, ?it/s]

  0%|          | 0/648 [00:01<?, ?it/s]

  0%|          | 0/648 [00:01<?, ?it/s]

/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/tor

['0.0186', '0.3269', '0.2984', '0.1347', '0.1619']
['0.9998', '0.9967', '0.9970', '0.9987', '0.9984']
['0.9988', '1.0000', '0.9990', '0.9997', '0.9995']
['0.9993', '0.9983', '0.9980', '0.9992', '0.9990']
['0.0112', '0.2670', '0.3411', '0.2509', '0.1594']
['0.9999', '0.9973', '0.9966', '0.9975', '0.9984']
['0.9986', '1.0000', '0.9994', '0.9998', '0.9996']
['0.9992', '0.9986', '0.9980', '0.9987', '0.9990']
['0.0166', '0.2072', '0.3449', '0.2192', '0.1957']
['0.9998', '0.9979', '0.9966', '0.9978', '0.9980']
['0.9987', '0.9998', '0.9992', '0.9994', '0.9998']
['0.9992', '0.9988', '0.9979', '0.9986', '0.9989']
['0.0093', '0.4097', '0.2558', '0.2007', '0.1933']
['0.9999', '0.9959', '0.9974', '0.9980', '0.9981']
['0.9986', '1.0000', '0.9993', '0.9997', '0.9997']
['0.9992', '0.9979', '0.9984', '0.9989', '0.9989']
['0.0127', '0.2808', '0.4186', '0.1822', '0.2271']
['0.9999', '0.9972', '0.9958', '0.9982', '0.9977']
['0.9985', '1.0000', '0.9992', '0.9994', '0.9998']
['0.9992', '0.9986', '0.9975', 

In [6]:
print('Final results')
total_results = {k: np.stack(v, axis=0) for k, v in total_results.items()}
for k, v in total_results.items():
    print(k, list("{0:0.4f}".format(i) for i in v.mean(axis=0)))

Final results
fnr ['0.0137', '0.2983', '0.3318', '0.1976', '0.1875']
rec ['0.9999', '0.9970', '0.9967', '0.9980', '0.9981']
pre ['0.9986', '0.9999', '0.9992', '0.9996', '0.9997']
f1 ['0.9992', '0.9985', '0.9979', '0.9988', '0.9989']


In [7]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [8]:
file = f'{args.pretrained_model}{bs}.json'
with open(f'../reports/{file}', 'w') as convert_file:
     convert_file.write(json.dumps(total_results, cls=NumpyEncoder))